# discriminated unions: sharing tuples or records?

Suppose we have:

In [ ]:
#!fsharp

type MyTuples =
    | One of a: string * b: string * c: string
    | Two of a: string * b: string * c: string * d: string
    | Three of a: string * b: string * c: string * e: string * f: int


Would it not be possible to share `a: string * b: string * c: string` among the cases of the DU?

In [ ]:
#!fsharp

type MyOne = One of a: string * b: string * c: string // the single-case DU

type MySharingTuples =
    | One of MyOne
    | Two of MyOne * d: string
    | Three of MyOne * e: string * f: int


It looks like the answer is _yes_ but we have made two completely different types. Trying to test the equality of `myTwo` and `mySharingTwo` will throw a compile-time `typecheck` error:

In [ ]:
#!fsharp

let myTwo = MyTuples.Two (a = "1", b = "2", c = "3", d = "4")

let mySharingTwo = MySharingTuples.Two (MyOne.One (a = "1", b = "2", c = "3"), d = "4")

// myTwo = mySharingTwo // compile-time typecheck error


In order to show any kind of equality relationship, say the one between `MyOne` and `MyTuples.One` we can launch into what looks like a fool’s errand:

In [ ]:
#!fsharp

let getMyOne t =
    match t with
    | MySharingTuples.One myOne -> myOne
    | MySharingTuples.Two (myOne, _) -> myOne
    | MySharingTuples.Three (myOne, _, _) -> myOne

let unwrapMyOne myOne =
    match myOne with
    | MyOne.One (a, b, c) -> (a, b, c)

let mySharingOne = mySharingTwo |> getMyOne |> unwrapMyOne


Notice that `mySharingOne` is of type `string * string * string` not of `a: string * b: string * c: string`. We can get the same type bound to `myTuplesOne`:

In [ ]:
#!fsharp

let unwrapMyTuplesOne t =
    match t with
    | MyTuples.One (a, b, c) -> (a, b, c)
    | MyTuples.Two (a, b, c, _) -> (a, b, c)
    | MyTuples.Three (a, b, c, _, _) -> (a, b, c)

let myTuplesOne = myTwo |> unwrapMyTuplesOne

Now that we have laboriously _transformed_ two different types into the same type we can express equality:

In [ ]:
#!fsharp

mySharingOne = myTuplesOne

True

Alternatively, when we want something of `a: string * b: string * c: string` we can use a conversion function, `toMyOne`:

In [ ]:
#!fsharp

let toMyOne t =
    match t with
    | MyTuples.One (a, b, c) -> MyOne.One(a = a, b = b, c = c)
    | MyTuples.Two (a, b, c, _) -> MyOne.One(a = a, b = b, c = c)
    | MyTuples.Three (a, b, c, _, _) -> MyOne.One(a = a, b = b, c = c)

(myTwo |> toMyOne) = (mySharingTwo |> getMyOne)

True

This is less laborious.

## what have i learned so far?

- this entire investigation of mine is possibly a waste of time
- it may not be possible to express tuples with field labels without a type qualifier (there is no such thing as an anonymous tuple with field labels in F#?)
- complex tuple-based structures would require nightmares of `match`-`with` decomposition
- replacing `match t with` with `function` confuses the compiler (under .NET Interactive)

And I have learned to ask, Is not the concept of casting from one type to another with tuples possible in F#? Today, I assume that `toMyOne` is the equivalent of casting that we would be looking for—when sent on this fool’s errand.

## the same fool’s errand but with records

What would this errand be like using anonymous records? Let’s look at `MyRecords`:

In [ ]:
#!fsharp

type MyRecords =
    | One of {| a: string; b: string; c: string |}
    | Two of {| a: string; b: string; c: string; d: string |}
    | Three of {| a: string; b: string; c: string; e: string; f: int |}

`{| a: string; b: string; c: string |}` is an [anonymous record](https://docs.microsoft.com/en-us/dotnet/fsharp/language-reference/anonymous-records) that can be upcast [📖 [docs](https://docs.microsoft.com/en-us/dotnet/fsharp/language-reference/casting-and-conversions)] to `obj` but downcast to nothing, including `MyRecordOne` below:

In [ ]:
#!fsharp

type MyRecordOne = { a: string; b: string; c: string }

type MySharingRecords =
    | One of MyRecordOne
    | Two of MyRecordOne * d: string // hey! …we are mixing records with tuples here!
    | Three of MyRecordOne * e: string * f: int

`MySharingRecords.Two` is mixing records with tuples which looks like I am perfecting confusion. At first, we might want to find the record equivalent of this record-tuple mix. Would it not be cool to do something like this in F#:

```fsharp
type MySharingRecords =
    | One of MyRecordOne
    | Two of { MyRecordOne with d: string }
    | Three of { MyRecordOne with e: string; f: int }
```

?

No? Yes? A language feature like this would make records essentially similar to dictionaries (which is what is going on in JavaScript) which means the pattern matching features of F# would have to change or be useless.

According to [Brian Berns](https://stackoverflow.com/users/344223/brian-berns), his StackOverflow [answer](https://stackoverflow.com/a/70544193/22944), mixing tuples with a _shared_ record is _exactly_ what we are supposed to do.

## not quite a fool’s errand: _mixing_ records and tuples

Mixing records and tuples in F# can be the equivalent of extending objects in JavaScript.

In [ ]:
#!fsharp

type MyTupleWithSharedRecord =
    | One of MyRecordOne
    | Two of MyRecordOne * d: string
    | Three of MyRecordOne * e: string * f: int

    with
    member this.UnwrapRecord =
        match this with
            | One record -> record
            | Two (record, _) -> record
            | Three (record, _, _) -> record

This `MyTupleWithSharedRecord.UnwrapRecord` is a static member similar to the other `unwrap*` functions in this writ.

In [ ]:
#!fsharp

let toAnonTwo (data: MyRecords) =
    match data with
    | MyRecords.Two anon -> anon
    | _ -> failwith "There has be another way better than this!"

let toMyRecordOne (data: {| a: string; b: string; c: string; d: string |}) =
    { a = data.a; b = data.b; c = data.c }

let myTwo = MyRecords.Two {| a = "1"; b = "2"; c = "3"; d = "4" |}

let mySharingTwo = MyTupleWithSharedRecord.Two ({ a = "1"; b = "2"; c = "3" }, d = "4")

(myTwo |> toAnonTwo |> toMyRecordOne) = mySharingTwo.UnwrapRecord

True

## what have i learned so far?

The most important take way here is that a record multiplied by ad hoc types in a tuple can approximate the object-extending features in JavaScript. An anonymous object (usually coming from JSON) can be converted to a local F# record type but embedding that record type in a discriminated union of tuples can capture any new properties hanging off the anonymous object that the local F# record type cannot (and should not) represent.

This `UnwrapRecord` member of the DU type can pass a local, formal F# record instance to some kind of storage layer. This is quite a flexible design!

@[BryanWilhite](https://twitter.com/BryanWilhite)